In [1]:
import json
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from tqdm.auto import tqdm

# Collecting Information from FITS Files

In [2]:
# visualisation for the extracted spectral data
def plot_spec_orders(spec_df_dict, plot_label, exp_filename):
    fig, axes = plt.subplots(
        40, 1, figsize=(8, 80), dpi=300, gridspec_kw={'hspace': .14}
    )

    for (spec_order, axis) in zip(
            sorted(spec_df_by_order_dict.keys(), reverse=True), axes):
        df = spec_df_dict[spec_order]

        axis.errorbar(
            df['wvl_angstrom'], y=df['flux'],
            yerr=df['flux_err'], fmt='o',
            c='tab:blue', ecolor='darkgrey', mfc='None',
            ms=1, elinewidth=.6, mew=.6
        )
        axis.text(
            s=f'Order {spec_order}', x=.03, y=.9,
            va='top', ha='left', transform=axis.transAxes,
            font='monospace', fontsize=8
        )
        axis.text(
            s=f'avg. flux/$\sigma_\mathrm{{flux}}$={np.nanmean(df["flux"] / df["flux_err"]):.1f}',
            x=.97, y=.9, va='top', ha='right', transform=axis.transAxes,
            font='monospace', fontsize=8
        )
        axis.axhline(0, c='k', lw=.4, ls=':', alpha=.8)
        axis.set_ylabel('Flux', font='monospace', fontsize=10)

        plot_upper_threshold = np.ceil(np.nanpercentile(df['flux'], 99) / 6) * 6
        y_major_locator_base = max(6, (plot_upper_threshold // 4) // 6 * 6)
        y_minor_locator_base = y_major_locator_base / 3
        y_upper_lim = np.ceil(plot_upper_threshold / y_minor_locator_base) * y_minor_locator_base
        axis.set_ylim(-y_minor_locator_base, y_upper_lim)
        axis.yaxis.set_major_locator(
            plt.MultipleLocator(base=y_major_locator_base, offset=y_minor_locator_base))
        axis.yaxis.set_minor_locator(
            plt.MultipleLocator(base=y_minor_locator_base))

        axis.set_xlim(np.floor(df['wvl_angstrom'].min() / 5) * 5 - 5,
                      np.ceil(df['wvl_angstrom'].max() / 5) * 5 + 5)
        axis.xaxis.set_major_locator(plt.MultipleLocator(base=25))
        axis.xaxis.set_minor_locator(plt.MultipleLocator(base=5))

        axis.tick_params(axis='both', right=True, top=True,
                         labelsize=8, direction='in', length=5, which='major')
        axis.tick_params(axis='both', right=True, top=True,
                         labelsize=8, direction='in', length=4, which='minor')

    axes[-1].set_xlabel(r'$\lambda$ [$\AA$]', font='monospace', fontsize=10)

    # set the title of the plot
    axes[0].set_title(f'{plot_label}', font='monospace', fontsize=12)
    fig.savefig(exp_filename, bbox_inches='tight')
    plt.close(fig)

In [3]:
obs_date = 230615
combined_img_fits_file_dir = f'data/raw-combined/{obs_date}/'
spec_fits_file_dir = f'data/extracted/{obs_date}/'

# path to plots and summary files
exp_path = f'results/'
fig_exp_path = os.path.join(exp_path, f'figures/{obs_date}')
file_exp_path = os.path.join(exp_path, f'spec_files/{obs_date}')

# create directories if not exist
for path in [exp_path, fig_exp_path, file_exp_path]:
    if not os.path.isdir(path):
        os.makedirs(path)

# full orders ranging from 299 to 338
full_orders = range(299, 339)

## Collection information from combined fits images

In [4]:
combined_img_info_dict = {}
combined_img_fits_filenames = [name for name in os.listdir(combined_img_fits_file_dir)
                               if re.search(r'combined(\d+-\d+)', name)]
for fits_img_filename in tqdm(sorted(
        combined_img_fits_filenames, key=lambda x: int(re.search(r'(\d+)-', x).group(1)))):
    print(f'processing \"{os.path.join(combined_img_fits_file_dir, fits_img_filename)}\"...')

    # import FITS data and header info
    with fits.open(os.path.join(combined_img_fits_file_dir, fits_img_filename)) as fits_img_file:
        fits_img_header = fits_img_file[0].header

    combined_img_info_dict[fits_img_filename] = {
        'src_img_fits_combined_img': json.dumps(
            re.findall(r'icm.*?\.a\.fits', str(fits_img_header['HISTORY']))),
    }

combined_img_info_df = pd.DataFrame(combined_img_info_dict).T

  0%|          | 0/20 [00:00<?, ?it/s]

processing "data/raw-combined/230615/combined1-6.fits"...
processing "data/raw-combined/230615/combined14-18.fits"...
processing "data/raw-combined/230615/combined19-22.fits"...
processing "data/raw-combined/230615/combined23-28.fits"...
processing "data/raw-combined/230615/combined29-32.fits"...
processing "data/raw-combined/230615/combined33-37.fits"...
processing "data/raw-combined/230615/combined45-47.fits"...
processing "data/raw-combined/230615/combined48-50.fits"...
processing "data/raw-combined/230615/combined51-55.fits"...
processing "data/raw-combined/230615/combined56-58.fits"...
processing "data/raw-combined/230615/combined66-70.fits"...
processing "data/raw-combined/230615/combined71-74.fits"...
processing "data/raw-combined/230615/combined75-77.fits"...
processing "data/raw-combined/230615/combined85-87.fits"...
processing "data/raw-combined/230615/combined93-96.fits"...
processing "data/raw-combined/230615/combined104-106.fits"...
processing "data/raw-combined/230615/com

## Collecting Information from FITS Files

In [5]:
spec_info_dict = {}
spec_fits_filenames = [name for name in os.listdir(spec_fits_file_dir)
                       if name.startswith('combined_spectra')]
for spec_fits_filename in tqdm(sorted(
        spec_fits_filenames, key=lambda x: int(re.search(r'(\d+)-', x).group(1)))):
    spec_label = re.sub(
        r'combined_spectra(\d+-\d+)', r'spec\1', spec_fits_filename.split('.')[0])
    print(f'processing \"{os.path.join(spec_fits_file_dir, spec_fits_filename)}\"...')

    # import FITS data and header info
    with fits.open(os.path.join(spec_fits_file_dir, spec_fits_filename)) as fits_file:
        fits_spec_data = fits_file[0].data
        fits_spec_header = fits_file[0].header

    orders = [int(s) for s in fits_spec_header['ORDERS'].split(',')]

    # extract spectral data from each order
    spec_df_by_order_dict = {}
    snr_by_order_dict = {}
    for idx, order in enumerate(orders):
        spec_df_by_order_dict[order] = pd.DataFrame({
            'wvl_um': fits_spec_data[idx, 0, :],
            'wvl_angstrom': fits_spec_data[idx, 0, :] * 1e4,
            'flux': fits_spec_data[idx, 1, :],
            'flux_err': fits_spec_data[idx, 2, :]
        })

        order_snr = (lambda x: x[~np.isnan(x)])(
            spec_df_by_order_dict[order]['flux'] / spec_df_by_order_dict[order]['flux_err']
        )
        snr_by_order_dict[order] = {
            'mean': np.mean(order_snr) if len(order_snr) > 0 else np.nan,
            'std': np.std(order_snr) if len(order_snr) > 0 else np.nan
        }

    # plot the spectral data
    plot_spec_orders(
        spec_df_by_order_dict,
        f'{obs_date}-{spec_label} ({fits_spec_header["OBJECT"]})',
        os.path.join(fig_exp_path, f'{obs_date}-{spec_label}.png')
    )

    # collect information from FITS header
    spec_info_dict[spec_fits_filename] = {
        'object': fits_spec_header['OBJECT'],
        'extracted_spec_fits': spec_fits_filename,
        'combined_img_fits': fits_spec_header['AIMAGE'],
        'src_img_fits_spec': json.dumps(
            re.findall(r'icm.*?\.a\.fits', str(fits_spec_header['HISTORY']))),
        'flat_fits': fits_spec_header['FLAT'],
        'wvl_cal_fits': fits_spec_header['WAVECAL'],
        'obs_date': obs_date,
        'obs_humidity': fits_spec_header['TCS_HUM'],
        'obs_air_temp_deg': fits_spec_header['TCS_AIRT'],
        'obs_mean_wind_speed_mph': fits_spec_header['TCS_WMSP'],
        'obs_wind_direction_deg': fits_spec_header['TCS_WDIR'],
        'obs_elevation_deg': fits_spec_header['TCS_EL'],
        'obs_azimuth_deg': fits_spec_header['TCS_AZ'],
        'obs_ra_hms_fk5': fits_spec_header['RA'],
        'obs_dec_hms_fk5': fits_spec_header['DEC'],
        'obs_avg_mjd': fits_spec_header['AVE_MJD'],
        'obs_total_exp_time': fits_spec_header['TOTITIME'],
        'obs_exp_time': fits_spec_header['ITIME'],
        'obs_avg_air_mass': fits_spec_header['AVE_AM'],
        'num_img_combined': fits_spec_header['NIMCOMB'],
        **{f'ap_pos_order{order}': fits_spec_header.get(f'APOSO{order}', None)
           for order in full_orders},
        **{f'extn_range_order{order}': json.dumps(
            list(map(int, fits_spec_header.get(f'XROR{order}', '0,0').split(','))))
            for order in full_orders},
        **{f'snr_mean_order{order}': snr_by_order_dict[order]['mean']
        if order in snr_by_order_dict else np.nan
           for order in full_orders},
        **{f'snr_std_order{order}': snr_by_order_dict[order]['std']
        if order in snr_by_order_dict else np.nan
           for order in full_orders}
    }

    # save the DataFrames as hdf5 files
    with pd.HDFStore(os.path.join(file_exp_path, f'{obs_date}-{spec_label}.h5'), 'w') as store:
        for order, spec_df in spec_df_by_order_dict.items():
            store[f'order{order}'] = spec_df

spec_info_df = pd.DataFrame(spec_info_dict).T

  0%|          | 0/20 [00:00<?, ?it/s]

processing "data/extracted/230615/combined_spectra1-6.fits"...
processing "data/extracted/230615/combined_spectra14-18.fits"...
processing "data/extracted/230615/combined_spectra19-22.fits"...
processing "data/extracted/230615/combined_spectra23-28.fits"...
processing "data/extracted/230615/combined_spectra29-32.fits"...
processing "data/extracted/230615/combined_spectra33-37.fits"...
processing "data/extracted/230615/combined_spectra45-47.fits"...
processing "data/extracted/230615/combined_spectra48-50.fits"...
processing "data/extracted/230615/combined_spectra51-55.fits"...
processing "data/extracted/230615/combined_spectra56-58.fits"...
processing "data/extracted/230615/combined_spectra66-70.fits"...
processing "data/extracted/230615/combined_spectra71-74.fits"...
processing "data/extracted/230615/combined_spectra75-77.fits"...
processing "data/extracted/230615/combined_spectra85-87.fits"...
processing "data/extracted/230615/combined_spectra93-96.fits"...
processing "data/extracted/

## Merging the Summary DataFrames

In [6]:
merged_summary_df = pd.merge(
    spec_info_df, combined_img_info_df, left_on='combined_img_fits', right_index=True, how='left')

# check if 'src_img_fits_spec' and 'src_img_fits_combined_img' are consistent, if available
merged_summary_df['is_src_img_fits_log_consistent'] = merged_summary_df.apply(
    lambda x: x['src_img_fits_spec'] == x['src_img_fits_combined_img'], axis=1)
merged_summary_df['src_img_fits'] = merged_summary_df.apply(
    lambda x: x['src_img_fits_spec'] if x['is_src_img_fits_log_consistent']
    else x['src_img_fits_combined_img'],
    axis=1)

merged_summary_df['group'] = merged_summary_df['object'].apply(
    lambda x: 'Telluric_STD' if 'Telluric' in x
    else 'RV_STD' if 'RVSTD' in x
    else f'LP_2442_gp{x[0]}'
)

merged_summary_df[[
    'src_img_fits', 'is_src_img_fits_log_consistent',
    'src_img_fits_spec', 'src_img_fits_combined_img'
]]

,src_img_fits,is_src_img_fits_log_consistent,src_img_fits_spec,src_img_fits_combined_img
combined_spectra1-6.fits,"[""icm.2023A023.230615.fname.00001.a.fits"", ""ic...",True,"[""icm.2023A023.230615.fname.00001.a.fits"", ""ic...","[""icm.2023A023.230615.fname.00001.a.fits"", ""ic..."
combined_spectra14-18.fits,"[""icm.2023A023.230615.fname.00014.a.fits"", ""ic...",False,[],"[""icm.2023A023.230615.fname.00014.a.fits"", ""ic..."
combined_spectra19-22.fits,NaN,False,"[""icm.2023A023.230615.fname.00019.a.fits"", ""ic...",NaN
combined_spectra23-28.fits,NaN,False,"[""icm.2023A023.230615.fname.00023.a.fits"", ""ic...",NaN
combined_spectra29-32.fits,NaN,False,"[""icm.2023A023.230615.fname.00029.a.fits"", ""ic...",NaN
combined_spectra33-37.fits,NaN,False,[],NaN
combined_spectra45-47.fits,NaN,False,[],NaN
combined_spectra48-50.fits,NaN,False,[],NaN
combined_spectra51-55.fits,NaN,False,[],NaN
combined_spectra56-58.fits,"[""icm.2023A023.230615.fname.00056.a.fits"", ""ic...",False,[],"[""icm.2023A023.230615.fname.00056.a.fits"", ""ic..."


In [7]:
# adjust columns order
merged_summary_df = merged_summary_df[[
    'group', 'object', 'extracted_spec_fits', 'combined_img_fits', 'flat_fits', 'wvl_cal_fits',
    'src_img_fits', 'is_src_img_fits_log_consistent', 'src_img_fits_spec', 'src_img_fits_combined_img',
    *[col for col in merged_summary_df.columns if 'obs_' in col], 'num_img_combined',
    *[col for col in merged_summary_df.columns if 'ap_pos_order' in col],
    *[col for col in merged_summary_df.columns if 'extn_range_order' in col],
    *[val for pair in zip(
        sorted([col for col in merged_summary_df.columns if 'snr_mean_order' in col]),
        sorted([col for col in merged_summary_df.columns if 'snr_std_order' in col]))
      for val in pair]
]]
merged_summary_df

,group,object,extracted_spec_fits,combined_img_fits,flat_fits,wvl_cal_fits,src_img_fits,is_src_img_fits_log_consistent,src_img_fits_spec,src_img_fits_combined_img,...,snr_mean_order334,snr_std_order334,snr_mean_order335,snr_std_order335,snr_mean_order336,snr_std_order336,snr_mean_order337,snr_std_order337,snr_mean_order338,snr_std_order338
combined_spectra1-6.fits,Telluric_STD,TelluricSTD,combined_spectra1-6.fits,combined1-6.fits,flat7-11.fits,wavecal12-13.fits,"[""icm.2023A023.230615.fname.00001.a.fits"", ""ic...",True,"[""icm.2023A023.230615.fname.00001.a.fits"", ""ic...","[""icm.2023A023.230615.fname.00001.a.fits"", ""ic...",...,132.971255,41.735509,133.661567,40.760635,132.268753,40.070592,127.108475,39.284463,118.297095,42.264053
combined_spectra14-18.fits,LP_2442_gp5,559c3847e65,combined_spectra14-18.fits,combined14-18.fits,flat7-11.fits,wavecal12-13.fits,"[""icm.2023A023.230615.fname.00014.a.fits"", ""ic...",False,[],"[""icm.2023A023.230615.fname.00014.a.fits"", ""ic...",...,46.231234,18.897741,45.179039,18.053243,44.030478,17.962211,39.826317,16.738143,31.113917,17.26133
combined_spectra19-22.fits,LP_2442_gp5,501cbf79576,combined_spectra19-22.fits,combined_spectra19-22.fits,flat7-11.fits,wavecal12-13.fits,NaN,False,"[""icm.2023A023.230615.fname.00019.a.fits"", ""ic...",NaN,...,44.877625,17.811921,43.890599,17.153313,42.750427,17.10801,39.151147,16.018455,31.316189,16.371122
combined_spectra23-28.fits,LP_2442_gp5,563d3c59db1,combined_spectra23-28.fits,combined_spectra23-28.fits,flat7-11.fits,wavecal12-13.fits,NaN,False,"[""icm.2023A023.230615.fname.00023.a.fits"", ""ic...",NaN,...,41.840172,17.968538,40.600881,17.127629,39.350394,17.04386,34.683506,15.395708,26.175485,16.709298
combined_spectra29-32.fits,LP_2442_gp2,2c620c86315,combined_spectra29-32.fits,combined_spectra29-32.fits,flat7-11.fits,wavecal12-13.fits,NaN,False,"[""icm.2023A023.230615.fname.00029.a.fits"", ""ic...",NaN,...,42.036547,16.900362,41.024821,16.389115,39.942302,16.160766,36.183616,15.141964,28.251501,15.70385
combined_spectra33-37.fits,Telluric_STD,TelluricSTD,combined_spectra33-37.fits,combined_spectra33-37.fits,flat7-11.fits,wavecal12-13.fits,NaN,False,[],NaN,...,140.342591,44.906867,141.724523,44.388851,140.257232,43.864893,137.644856,43.870178,129.984569,46.039342
combined_spectra45-47.fits,LP_2442_gp5,5a414c74fee,combined_spectra45-47.fits,combined_spectra45-47.fits,flat38-42.fits,wavecal43-44.fits,NaN,False,[],NaN,...,62.676899,22.079735,62.398522,21.4107,61.628397,21.767497,58.834135,20.805522,51.679282,21.943074
combined_spectra48-50.fits,LP_2442_gp2,25661373b6e,combined_spectra48-50.fits,combined_spectra48-50.fits,flat38-42.fits,wavecal43-44.fits,NaN,False,[],NaN,...,32.070329,13.302494,31.056489,12.671622,29.971639,12.585006,27.19876,11.598383,21.219591,12.345532
combined_spectra51-55.fits,LP_2442_gp2,2afb04570cb,combined_spectra51-55.fits,combined_spectra51-55.fits,flat38-42.fits,wavecal43-44.fits,NaN,False,[],NaN,...,31.455264,13.929453,30.896105,13.308516,30.073961,13.387663,27.111755,11.976359,21.148789,13.258377
combined_spectra56-58.fits,LP_2442_gp2,20e00a62ae2,combined_spectra56-58.fits,combined56-58.fits,flat38-42.fits,wavecal43-44.fits,"[""icm.2023A023.230615.fname.00056.a.fits"", ""ic...",False,[],"[""icm.2023A023.230615.fname.00056.a.fits"", ""ic...",...,26.06798,11.404086,25.556062,10.942409,24.987232,10.912346,22.814232,10.039584,18.229189,10.467942


In [8]:
# save the collected information as CSV file
merged_summary_df.to_csv(f'results/{obs_date}-summary.csv', index=False)